# Generate report from exprted photometer data  

Use raw folder with photometer exported data

## Imports

In [ ]:
VERBOSE_NOTEBOOK = False
WARNING_DISABLE = True
DEBUG = False

In [ ]:
from os import path
import warnings
from scipy.optimize import OptimizeWarning

if WARNING_DISABLE:
    warnings.simplefilter('ignore', RuntimeWarning)
    warnings.simplefilter('ignore', OptimizeWarning)
    warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [ ]:
from elisarep.mkinout import make_input_paths, parse_dir_name

# ANALYSIS_DIR = './reports/export/230801_AAV9-ELISA_sey_GN004240-053'
ANALYSIS_DIR = 'c:/work/hamilton/analysis/230801_AAV9-ELISA_sey_GN004240-053'
CONFIG_DIR = './data'

PARSED_DIR = parse_dir_name(ANALYSIS_DIR)

input_files = make_input_paths(ANALYSIS_DIR)
WORKLIST_FILE_PATH = input_files['worklist']
PARAMS_FILE_PATH = input_files['params']

In [ ]:
from dataclasses import dataclass
import json


@dataclass
class Config:
    def __init__(self,
                 pandoc_bin: path,
                 pdflatex_bin: path,
                 reference_docx: path,
                 data_dir: path = "./data",
                 plate_layout_id: path = "./data/plate_layout_ident.csv",
                 plate_layout_num: path = "./data/plate_layout_num.csv",
                 plate_layout_dil_id: path = "./data/plate_layout_dil_id.csv",
                 numeric_warning_disable: bool = True
                 ):
        self.pandoc_bin = pandoc_bin
        self.pdflatex_bin = pdflatex_bin
        self.reference_docx = reference_docx
        self.data_dir = data_dir
        self.plate_layout_id = plate_layout_id
        self.plate_layout_num = plate_layout_num
        self.plate_layout_dil_id = plate_layout_dil_id
        self.numeric_warning_disable = numeric_warning_disable


def read_config(cfg_path):
    with open(path.join(CONFIG_DIR, "config.json")) as json_file:
        jd = json.load(json_file)

    return Config(jd['pandoc_bin'], jd['pdflatex_bin'], jd['reference_docx'])


cfg = read_config('./data/config.json')

## Layouts

In [ ]:
from elisarep.readdata import read_layouts

PLATE_LAYOUT_ID = 'plate_layout_ident.csv'
PLATE_LAYOUT_NUM = 'plate_layout_num.csv'
PLATE_LAYOUT_DIL_ID = 'plate_layout_dil_id.csv'


g_lay = read_layouts(path.join(CONFIG_DIR, PLATE_LAYOUT_ID),
                     path.join(CONFIG_DIR, PLATE_LAYOUT_NUM),
                     path.join(CONFIG_DIR, PLATE_LAYOUT_DIL_ID))

if VERBOSE_NOTEBOOK:
    display(g_lay)

## Worklist

In [ ]:
from elisarep.worklist import predil_worklist, check_worklist
from elisarep.readdata import read_params

wl_raw = predil_worklist(WORKLIST_FILE_PATH)
params = read_params(PARAMS_FILE_PATH)

## Dilution to Concentration

Define dilution dataframe. The dataframe is indexed according plate layout, index of refference dataframe corresponds to refference of the `plate_layout_dil`.

In [ ]:
# TODO: read reference value from parameters
from elisarep.sample import make_concentration
REF_VAL_MAX = 1.7954e+10
DILUTIONS = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0]

reference_conc = make_concentration(REF_VAL_MAX, DILUTIONS)

if VERBOSE_NOTEBOOK:
    display(reference_conc)

## Report generation

In [ ]:
from elisarep.config import config as cfg
from elisarep.config import init_config

init_config(ANALYSIS_DIR, CONFIG_DIR)

In [ ]:
from elisarep.reportgen import gen_report_raw

reports = gen_report_raw(wl_raw, params, g_lay,
                         reference_conc, ANALYSIS_DIR)

In [ ]:
from elisarep.mdhandling import md2docx, md2pdf, export_main_report
from elisarep.reportmd import save_md

export_main_report(reports, ANALYSIS_DIR, cfg['pandoc_bin'],
                   cfg['reference_docx'])

docxr = True
pdf = True
for report in reports:
    print('Report for plate {} saved as {}'.format(
        report['plate'], report['path']))
    save_md(report['path'], report['md'])

    # TODO: save unformatted results
    xlsx_file = path.splitext(report['path'])[0] + '_results.xlsx'
    report['df'].to_excel(xlsx_file)

    if docxr:
        md2docx(cfg['pandoc_bin'], cfg['reference_docx'], report['path'])
    if pdf:
        md2pdf(cfg['pandoc_bin'], cfg['pdflatex_bin'], report['path'])

print('Done.')

In [ ]:
from elisarep.reportmain import check_report_crc

CHECK_REPORT_CRC = True
REPORT_PLATES_CRC = [2535847545, 3489686272, 2582870018]
if CHECK_REPORT_CRC:
    for report, crc in zip(reports, REPORT_PLATES_CRC):
        try:
            check_report_crc(report['md'], crc)
        except Exception as e:
            print('{} for {}'.format(e, report['path']))

Uncomment the next block to build the module

In [ ]:
# ! python -m build --sdist --wheel

In [ ]:
import elisarep.layouthandle as hl
import data.layouts as ly

hl.to_matrix(ly.l_plate_layout_ident, 8)

In [ ]:
l1 = [['a', 1], ['b', 2], ['c', 3]]
l2 = [['a', 1], ['b', 2], ['c', 3]]

l1 == l2

## `pip install`

In [ ]:
# ! python.exe -m pip install --upgrade pip

In [ ]:
# ! python.exe -m pip install --upgrade pandas

In [ ]:
# ! pip install mypy

In [ ]:
# ! python .\report_gen.py --help

In [ ]:
# ! python.exe -m pip install pip install python-docx

# Wowrd

In [ ]:
from docx.enum.style import WD_STYLE_TYPE
import docx
from docx import Document

document = Document()
styles = document.styles
styles = document.styles
paragraph_styles = [
    s for s in styles if s.type == WD_STYLE_TYPE.PARAGRAPH
]
for style in paragraph_styles:
    print(style.name)

In [ ]:
import re

r = re.compile(r'(\*\*).*?(\*\*)')

str = '**<2.805e+9**'
m = r.match(str)
print(m)

In [ ]:
re.sub('[\*\*]', '', str)

# MS Word replace

In [ ]:
text = '''
Fabry, AAV9-Y
PFB,	P-Y
EFB,	E-Y
Pompe,	AAV9-E
PPO,	P-E
EPO,	E-E
rE-Ert,	report
PP648,	PRPL1
PP073,	PRPL2
PP5786,	PRPL2
PHCP,	PHCP1
'''

data = {}
lines = text.strip().split('\n')
for line in lines:
    key, value = line.split(',')
    data[key.strip()] = value.strip()

print(data)

In [ ]:
from docx import Document

# Path to the Word document
doc_path = "C:/work/elisa-report/media/DMD-212109_HEK293-HostCellProtein_ELISA.docx"
print(doc_path)
print(f'Replacing by\n  {data}')

# Read the document
doc = Document(doc_path)

# Replace strings
replaced = data.copy()
verbose = False
for key, value in replaced.items():
    replaced[key] = 0
for paragraph in doc.paragraphs:
    for key, value in data.items():
        if key in paragraph.text:
            txt_old = paragraph.text
            paragraph.text = paragraph.text.replace(key, value)
            replaced[key] += 1
            print(f'{txt_old} -> {paragraph.text}') if verbose else None

for table in doc.tables:
    for row in table.rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                for key, value in data.items():
                    if key in paragraph.text:
                        txt_old = paragraph.text
                        paragraph.text = paragraph.text.replace(key, value)
                        replaced[key] += 1
                        print(
                            f'{txt_old} -> {paragraph.text}') if verbose else None


print(f'Replacements:\n  {replaced}')

# Save the modified document
new_doc_path = doc_path.replace('.docx', '_new.docx')
doc.save(new_doc_path)
print(f'New docx file saved to {new_doc_path}')